## 运行环境

In [1]:
%load_ext watermark
%watermark

2017-04-09T09:06:42+08:00

CPython 3.5.2
IPython 5.3.0

compiler   : GCC 4.9.2
system     : Linux
release    : 3.16.0-4-amd64
machine    : x86_64
processor  : 
CPU cores  : 4
interpreter: 64bit


In [2]:
import tensorflow as tf
print(tf.__version__)

1.0.1


## 构建一个词的 Word Embedding
在构建统计语言模型的时候，我们把所有看到的词都放入了 dict 里面，这时候影响不大，因为每个词占用的内存较小。

在使用神经网络做自然语言处理的时候，我们一般都会对词表做一个截断操作，取最高频的 n 个（也有人按词频阈值做截断）。这样有两个好处：
1. 减少模型的内存使用。
2. 只出现过一两次的词，在整个优化过程中往往也很难学好。不如把这些词直接全看成未登录词。

In [3]:
tf.reset_default_graph()

In [4]:
# 决定了 embedding 的维度 （隐层节点数）
word_embedding_dim = 128
# 决定了词表数量, 预留一个未登录词
vocab_size = 80000 + 1
UNK_IDX = 0

# 这里需要把 Word embedding 放到 Variable 里面。因为 Word embedding 是要随机初始化，跟着数据不断变化的。
# 它相当于普通神经网络中的权重。

# 在梯度下降时， tensorflow 的 Optimizer 会自动找到 Graph 中的 Variable，计算梯度并进行更新。
word_embedding = tf.Variable(tf.random_uniform([vocab_size, word_embedding_dim]))

# placeholder 和 variable 基本都可以当做 Tensor 来用
# 注意这里的输入是 int32 类型，表示一个词 ID。这里我们需要对数据进行预处理，以把高频词映射到 [1, 80000] 之间，不在词表里面的词设置成 UNK, ID 为 0
# 这里我们假设输入是两个词

# 这里 Shape 的第一维我们指定为 None，是表示第一维可以根据数据进行变化，因此同样一个程序可以适应梯度下降时不同的 batch_size
input_data = tf.placeholder(tf.int32, shape=[None, 2], name='input_data')

input_embeds = tf.nn.embedding_lookup(word_embedding, input_data)

因为 input_data 是一个二维矩阵，lookup 之后得到的其实是一个『三维的矩阵』

怎么理解呢？如果我们一个样本有两个词，拿到的就是矩阵的两行，因此是一个矩阵

但是我们同时有多个样本，因此需要对这个矩阵再『扩张』一个维度。而这个维度因为数据还未给出，大小是未知的

In [5]:
input_embeds

<tf.Tensor 'embedding_lookup:0' shape=(?, 2, 128) dtype=float32>

### 两个词的向量做相加

In [6]:
# reduce 开头的函数一般有一个 axis 参数，决定按行、按列或者按整个矩阵进行 reduce
context_embeds = tf.reduce_sum(input_embeds, axis=1)

In [7]:
# 注意观察 context_embds 的 shape
# 因为 placeholder 第一位的维度是 None，这里 TF 没法确切知道第一维最后的 shape
context_embeds

<tf.Tensor 'Sum:0' shape=(?, 128) dtype=float32>

### 相加的词向量再映射到 N 个词的概率分布

In [42]:
# 激活之前的输出
raw_output = tf.layers.dense(context_embeds, 2)

In [43]:
# 加 softmax 之后的输出
output = tf.nn.softmax(raw_output)

In [44]:
output

<tf.Tensor 'Softmax_1:0' shape=(?, 2) dtype=float32>

In [10]:
import numpy as np

In [45]:
with tf.Session() as sess:
    # 因为引入了 variable，所以需要进行初始化
    sess.run(tf.global_variables_initializer())
    # 输出的矩阵比较大，我们只看前 10 列
    print(sess.run(output, feed_dict={input_data: np.asarray([[1, 2]])})[:, :10])

[[ 0.99105352  0.0089465 ]]


- 同第三讲示例代码中的类似 (sigmoid + cross entropy)，softmax 配合 cross entropy 的时候，在求导时两个连着看，也可以做分母的消除，因此在计算 cost 的时候我们要把 raw_output 喂给 tf 的这个损失函数

In [70]:
# 样本的 labels 也需要用 placeholder 放置
labels = tf.placeholder(tf.int32, shape=[None, 2], name='labels')

# 因为我们每个样本的 label 只有一个，使用稠密的 softmax 算 cost 及求导太浪费了。这里使用 sparse 版本即可。
# 如果你的 label 是完整的 N 个词上的概率分布，这时候可以使用 tf.nn.softmax_cross_entropy_with_logits
# cost = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=raw_output, labels=labels)
cost = tf.nn.softmax_cross_entropy_with_logits(logits=raw_output, labels=labels)

In [71]:
raw_output

<tf.Tensor 'dense_2/BiasAdd:0' shape=(?, 2) dtype=float32>

In [72]:
labels

<tf.Tensor 'labels_5:0' shape=(?, 2) dtype=int32>

In [76]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    dummy_feed_dict = {input_data: np.asarray([[1, 2], [4, 3]]),
                       labels: np.asarray([[1, 0], [0, 1]])}
    for i in range(100):
        sess.run(train_step, feed_dict=dummy_feed_dict)
        if i % 20 == 0:
            print("Iteration %d" % i)
            print("Cost: %f" % cost.eval(feed_dict=dummy_feed_dict)[0])
            # 查看输出中 ID == 3 的概率
            print("Probability: %f" % output.eval(feed_dict=dummy_feed_dict)[0, 0])
            print("Probability: %f" % output.eval(feed_dict=dummy_feed_dict)[1, 0])
            output_tst = output.eval(feed_dict=dummy_feed_dict)
#             print("The ouput shape is {}".format(output.eval(feed_dict=dummy_feed_dict).shape))
            print("------")

Iteration 0
Cost: 0.293557
Probability: 0.745607
Probability: 0.291226
------
Iteration 20
Cost: 0.083625
Probability: 0.919776
Probability: 0.086740
------
Iteration 40
Cost: 0.046348
Probability: 0.954709
Probability: 0.048951
------
Iteration 60
Cost: 0.031678
Probability: 0.968818
Probability: 0.033695
------
Iteration 80
Cost: 0.023928
Probability: 0.976356
Probability: 0.025547
------


In [77]:
output_tst

array([[ 0.97635603,  0.02364396],
       [ 0.02554693,  0.97445315]], dtype=float32)

In [75]:
output_a

array([ 0.02180748,  0.97819251], dtype=float32)

In [61]:
output_a.argmax()

1

In [39]:
output_a[5]

0.94072884

## 作业

请根据这个基本的框架程序进行扩展，使用你的语料库进行训练。并完成 3 个名词各自最相近的 Top 10 个词的检索

In [14]:
np.asarray([[1, 2]])

array([[1, 2]])

In [15]:
np.arange(16).reshape((1, 16))[0, 3]

3

In [16]:
np.asarray([[3], [4]])

array([[3],
       [4]])